In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, SpectralClustering
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (20, 15)

## 读取数据

In [3]:
songs = pd.read_csv('all_songs_info.csv', index_col='song')
songs.head(2)

,attached_pic,album,artist,avg_frame_rate,bit_rate,bits_per_raw_sample,channel_layout,channels,chroma_location,codec_long_name,...,start_time,time_base,width,lyricist,composer,lyric,arranger,release_date,distributor,lyric_len
song,,,,,,,,,,,,,,,,,,,,,
可爱女人,1,Jay,周杰伦,0/0,321289,8.0,stereo,2,center,Motion JPEG,...,0.025056,1/90000,445.0,徐若瑄,周杰伦,作词：徐若瑄\n\n作曲：周杰伦\n\n编曲：周杰伦\n\n想要有直升机 想要和妳飞到宇宙去...,周杰伦,2000-11-06,阿尔发音乐,833
完美主义,1,Jay,周杰伦,0/0,321260,8.0,stereo,2,center,Motion JPEG,...,0.025056,1/90000,445.0,方文山,周杰伦,作词：方文山\n\n作曲：周杰伦\n\n编曲：洪敬尧\n\n如果说怀疑 可以造句 如果说分离...,洪敬尧,2000-11-06,阿尔发音乐,552


In [4]:
def cut_lyric(lyric):
    if lyric is not np.nan:
        words = jieba.cut(lyric)
        words = [word for word in words if word not in ['\n', '\t', ' ']]
        words = ' '.join(words)
        return words

In [5]:
cut_lyrics = songs.lyric.apply(cut_lyric)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.948 seconds.
Prefix dict has been built succesfully.


In [6]:
len(cut_lyrics)

180

In [7]:
cut_lyrics.isnull().sum()

26

In [8]:
cut_lyrics.dropna(inplace=True)

In [9]:
cut_lyrics.shape

(154,)

In [10]:
embed = hub.Module('https://tfhub.dev/google/nnlm-zh-dim50/1')

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/0483635b6c375d7fb6c7dc388b12a5ae3fa2470e/variables/variables' with embeddings


In [11]:
sess = tf.InteractiveSession()

In [12]:
songnames = cut_lyrics.index.tolist()

In [13]:
words = cut_lyrics.tolist()

In [14]:
embeddings = embed(words)

In [15]:
sess.run([tf.global_variables_initializer(), tf.tables_initializer()])

[None, None]

In [16]:
embeddings = embeddings.eval()

In [17]:
embeddings.shape

(154, 50)

## 未聚类之前的分布

In [18]:
pca = PCA(n_components=2)

In [ ]:
embeddings_2d_pca = pca.fit_transform(embeddings)